In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
import pymongo

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [3]:
scrapable = ['Processors', 'Chipsets']
scrape = scrapable[0]

In [4]:
url = 'https://ark.intel.com/content/www/us/en/ark.html#@'+scrape
csvfilename = f'IntelArk{scrape}.csv'
baseurl = 'https://ark.intel.com/'

browser.visit(url)

rootHTML = bs(browser.html, 'html.parser')

In [5]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [6]:
db = client.IntelScrape
collection = db.items

In [7]:
for div in rootHTML.find_all('div', scrape):
    for link in rootHTML.\
        find_all('div', {'data-parent-panel-key':div['data-panel-key']})[0].\
        find_all('a'):
        browser.visit(baseurl+link['href'])
        time.sleep(1)
        for td in bs(browser.html, 'html.parser').find_all('td', class_='ark-product-name'):
            browser.visit(baseurl+td.a['href'])
            time.sleep(1)
            data = {'scrape':scrape}
            try:
                for c in bs(browser.html, 'html.parser').\
                    find_all('div', class_='arkProductSpecifications')[0].\
                    find_all('span', class_=['label','value']):
                    if c['class'][0] == 'label':
                        temp = c.text.strip().replace('.','_')
                    else:
                        data[temp] = c.text.strip()
            except:
                print('failed '+link['href'])
                continue
            collection.insert_one(data)
            print(td.a.text, end='; ')

Intel® Core™ i9-10980XE Extreme Edition Processor; Intel® Core™ i9-10940X X-series Processor; Intel® Core™ i9-10920X X-series Processor; Intel® Core™ i9-10900X X-series Processor; Intel® Core™ i9-9980XE Extreme Edition Processor; Intel® Core™ i9-9960X X-series Processor; Intel® Core™ i9-9940X X-series Processor; Intel® Core™ i9-9920X X-series Processor; Intel® Core™ i9-9900X X-series Processor; Intel® Core™ i9-9820X X-series Processor; Intel® Core™ i7-9800X X-series Processor; Intel® Core™ i9-7980XE Extreme Edition Processor; Intel® Core™ i9-7960X X-series Processor; Intel® Core™ i9-7940X X-series Processor; Intel® Core™ i9-7920X X-series Processor; Intel® Core™ i9-7900X X-series Processor; Intel® Core™ i7-7820X X-series Processor; Intel® Core™ i7-7800X X-series Processor; Intel® Core™ i7-7740X X-series Processor; Intel® Core™ i5-7640X X-series Processor; Intel® Core™ i7-6950X Processor Extreme Edition; Intel® Core™ i7-6900K Processor; Intel® Core™ i7-6850K Processor; Intel® Core™ i7-6

Intel® Core™ i7-6700HQ Processor; Intel® Core™ i7-6660U Processor; Intel® Core™ i7-6650U Processor; Intel® Core™ i7-6600U Processor; Intel® Core™ i7-6567U Processor; Intel® Core™ i7-6560U Processor; Intel® Core™ i7-6500U Processor; Intel® Core™ i5-6600 Processor; Intel® Core™ i5-6685R Processor; Intel® Core™ i5-6600K Processor; Intel® Core™ i5-6600T Processor; Intel® Core™ i5-6585R Processor; Intel® Core™ i5-6500 Processor; Intel® Core™ i5-6500T Processor; Intel® Core™ i5-6500TE Processor; Intel® Core™ i5-6402P Processor; Intel® Core™ i5-6400 Processor; Intel® Core™ i5-6400T Processor; Intel® Core™ i5-6440EQ Processor; Intel® Core™ i5-6440HQ Processor; Intel® Core™ i5-6442EQ Processor; Intel® Core™ i5-6360U Processor; Intel® Core™ i5-6350HQ Processor; Intel® Core™ i5-6300HQ Processor; Intel® Core™ i5-6300U Processor; Intel® Core™ i5-6200U Processor; Intel® Core™ i5-6260U Processor; Intel® Core™ i5-6267U Processor; Intel® Core™ i5-6287U Processor; Intel® Core™ i3-6300 Processor; Intel® 

Intel® Core™ i7-3840QM Processor; Intel® Core™ i7-3820QM Processor; Intel® Core™ i7-3770T Processor; Intel® Core™ i7-3770S Processor; Intel® Core™ i7-3770K Processor; Intel® Core™ i7-3770 Processor; Intel® Core™ i7-3740QM Processor; Intel® Core™ i7-3720QM Processor; Intel® Core™ i7-3689Y Processor; Intel® Core™ i7-3687U Processor; Intel® Core™ i7-3667U Processor; Intel® Core™ i7-3635QM Processor; Intel® Core™ i7-3632QM Processor; Intel® Core™ i7-3632QM Processor; Intel® Core™ i7-3630QM Processor; Intel® Core™ i7-3615QM Processor; Intel® Core™ i7-3615QE Processor; Intel® Core™ i7-3612QM Processor; Intel® Core™ i7-3612QM Processor; Intel® Core™ i7-3612QE Processor; Intel® Core™ i7-3610QM Processor; Intel® Core™ i7-3610QE Processor; Intel® Core™ i7-3555LE Processor; Intel® Core™ i7-3540M Processor; Intel® Core™ i7-3537U Processor; Intel® Core™ i7-3520M Processor; Intel® Core™ i7-3517UE Processor; Intel® Core™ i7-3517U Processor; Intel® Core™ i5-3610ME Processor; Intel® Core™ i5-3570S Proc

Intel® Core™2 Duo Processor E6700; Intel® Core™2 Duo Processor E6600; Intel® Core™2 Duo Processor E6550; Intel® Core™2 Duo Processor E6540; Intel® Core™2 Duo Processor E6420; Intel® Core™2 Duo Processor E6320; Intel® Core™2 Duo Processor E6400; Intel® Core™2 Duo Processor E6300; Intel® Core™2 Duo Processor E4700; Intel® Core™2 Duo Processor E4600; Intel® Core™2 Duo Processor E4500; Intel® Core™2 Duo Processor E4400; Intel® Core™2 Duo Processor E4300; Intel® Core™2 Duo Processor L7700; Intel® Core™2 Duo Processor L7500; Intel® Core™2 Duo Processor L7400; Intel® Core™2 Duo Processor L7300; Intel® Core™2 Duo Processor L7200; Intel® Core™ Duo Processor L2400; Intel® Core™ Duo Processor L2300; Intel® Core™ Duo Processor L2500; Intel® Core™2 Duo Processor P9700; Intel® Core™2 Duo Processor P9600; Intel® Core™2 Duo Processor P9500; Intel® Core™2 Duo Processor P8800; Intel® Core™2 Duo Processor P8700; Intel® Core™2 Duo Processor P8600; Intel® Core™2 Duo Processor P8400; Intel® Core™2 Duo Proce

Intel® Pentium® Processor G630; Intel® Pentium® Processor G622; Intel® Pentium® Processor G620T; Intel® Pentium® Processor G620; Intel® Pentium® Processor G645T; Intel® Pentium® Processor P6300; Intel® Pentium® Processor P6200; Intel® Pentium® Processor P6100; Intel® Pentium® Processor P6000; Intel® Pentium® Processor T4500; Intel® Pentium® Processor T4400; Intel® Pentium® Processor T4300; Intel® Pentium® Processor T4200; Intel® Pentium® Processor T3400; Intel® Pentium® Processor T3200; Intel® Pentium® Processor T2390; Intel® Pentium® Processor T2370; Intel® Pentium® Processor T2330; Intel® Pentium® Processor T2310; Intel® Pentium® Processor T2130; Intel® Pentium® Processor T2080; Intel® Pentium® Processor T2060; Intel® Pentium® Processor U5600; Intel® Pentium® Processor U5400; Intel® Pentium® 4 Processor 745 supporting HT Technology; Intel® Pentium® 4 Processor 672 supporting HT Technology; Intel® Pentium® 4 Processor 670 supporting HT Technology; Intel® Pentium® 4 Processor 662 suppo

...

Intel® Xeon® Gold 6126F Processor; Intel® Xeon® Gold 6126 Processor; Intel® Xeon® Gold 5122 Processor; Intel® Xeon® Gold 5120T Processor; Intel® Xeon® Gold 5120 Processor; Intel® Xeon® Gold 5119T Processor; Intel® Xeon® Gold 5118 Processor; Intel® Xeon® Gold 5115 Processor; Intel® Xeon® Silver 4116T Processor; Intel® Xeon® Silver 4116 Processor; Intel® Xeon® Silver 4114T Processor; Intel® Xeon® Silver 4114 Processor; Intel® Xeon® Silver 4112 Processor; Intel® Xeon® Silver 4110 Processor; Intel® Xeon® Silver 4109T Processor; Intel® Xeon® Silver 4108 Processor; Intel® Xeon® Bronze 3106 Processor; Intel® Xeon® Bronze 3104 Processor; Intel® Xeon® Processor E7-8894 v4; Intel® Xeon® Processor E7-8893 v4; Intel® Xeon® Processor E7-8891 v4; Intel® Xeon® Processor E7-8890 v4; Intel® Xeon® Processor E7-8880 v4; Intel® Xeon® Processor E7-8870 v4; Intel® Xeon® Processor E7-8867 v4; Intel® Xeon® Processor E7-8860 v4; Intel® Xeon® Processor E7-4850 v4; Intel® Xeon® Processor E7-4830 v4; Intel® Xeon®

Intel® Xeon® Processor E5-2430; Intel® Xeon® Processor E5-4640; Intel® Xeon® Processor E5-2430L; Intel® Xeon® Processor E5-4650; Intel® Xeon® Processor E5-2630; Intel® Xeon® Processor E5-2440; Intel® Xeon® Processor E5-2448L; Intel® Xeon® Processor E5-2630L; Intel® Xeon® Processor E5-4650L; Intel® Xeon® Processor E5-2450; Intel® Xeon® Processor E5-2637; Intel® Xeon® Processor E5-2640; Intel® Xeon® Processor E5-2450L; Intel® Xeon® Processor E5-2470; Intel® Xeon® Processor E5-2643; Intel® Xeon® Processor E5-2648L; Intel® Xeon® Processor E5-2650; Intel® Xeon® Processor E5-2650L; Intel® Xeon® Processor E5-2658; Intel® Xeon® Processor E5-2660; Intel® Xeon® Processor E5-2665; Intel® Xeon® Processor E5-2667; Intel® Xeon® Processor E5-2670; Intel® Xeon® Processor E5-2680; Intel® Xeon® Processor E5-2687W; Intel® Xeon® Processor E5-2690; Intel® Xeon® Processor E3-1535M v6; Intel® Xeon® Processor E3-1505M v6; Intel® Xeon® Processor E3-1505L v6; Intel® Xeon® Processor E3-1501L v6; Intel® Xeon® Pro

Intel® Xeon® Processor 7120N; Intel® Xeon® Processor 7120M; Intel® Xeon® Processor 7110N; Intel® Xeon® Processor 7110M; Intel® Xeon® Processor 7041; Intel® Xeon® Processor 7040; Intel® Xeon® Processor 7020; Intel® Xeon® Processor 7030; Intel® Xeon® Processor X6550; Intel® Xeon® Processor E6540; Intel® Xeon® Processor E6510; Intel® Xeon® Processor X5690; Intel® Xeon® Processor X5687; Intel® Xeon® Processor X5680; Intel® Xeon® Processor X5677; Intel® Xeon® Processor X5675; Intel® Xeon® Processor X5672; Intel® Xeon® Processor X5670; Intel® Xeon® Processor X5667; Intel® Xeon® Processor X5660; Intel® Xeon® Processor X5650; Intel® Xeon® Processor E5649; Intel® Xeon® Processor X5647; Intel® Xeon® Processor E5645; Intel® Xeon® Processor L5640; Intel® Xeon® Processor E5640; Intel® Xeon® Processor L5638; Intel® Xeon® Processor L5630; Intel® Xeon® Processor E5630; Intel® Xeon® Processor E5620; Intel® Xeon® Processor L5618; Intel® Xeon® Processor L5609; Intel® Xeon® Processor E5607; Intel® Xeon® P

Intel® Xeon® Processor 1.60 GHz, 1M Cache, 400 MHz FSB; Intel® Xeon® Processor 1.50 GHz, 1M Cache, 400 MHz FSB; Intel® Xeon® Processor 1.50 GHz, 512K Cache, 400 MHz FSB; Intel® Xeon® Processor 1.50 GHz, 256K Cache, 400 MHz FSB; Intel® Xeon® Processor 1.40 GHz, 512K Cache, 400 MHz FSB; Intel® Xeon® Processor 1.40 GHz, 256K Cache, 400 MHz FSB; Intel® Pentium® III Xeon® Processor 1.00 GHz, 256K Cache, 133 MHz FSB; Intel® Pentium® III Xeon® Processor 933 MHz, 256K Cache, 133 MHz FSB; Intel® Pentium® III Xeon® Processor 900 MHz, 2M Cache, 100 MHz FSB; Intel® Pentium® III Xeon® Processor 866 MHz, 256K Cache, 133 MHz FSB; Intel® Pentium® III Xeon® Processor 800 MHz, 256K Cache, 133 MHz FSB; Intel® Pentium® III Xeon® Processor 733 MHz, 256K Cache, 133 MHz FSB; Intel® Pentium® III Xeon® Processor 700 MHz, 2M Cache, 100 MHz FSB; Intel® Pentium® III Xeon® Processor 700 MHz, 1M Cache, 100 MHz FSB; Intel® Pentium® III Xeon® Processor 667 MHz, 256K Cache, 133 MHz FSB; Intel® Pentium® III Xeon® Proce

Intel® Quark™ SoC X1011; Intel® Quark™ SoC X1020; Intel® Quark™ SoC X1020D; Intel® Quark™ SoC X1021; Intel® Quark™ SoC X1021D; Intel® Quark™ Microcontroller D1000; Intel® Quark™ Microcontroller D2000; Intel® Quark™ SE C1000 Microcontroller; 

In [8]:
browser.quit()

In [9]:
import csv
with open(csvfilename,'w',newline='') as csvfile:
    cols = []
    data = db.items.find({'scrape':{'$eq':scrape}})
    for i in data:
        cols.extend([x for x in i.keys() if x not in cols])
    cols.remove('_id')
    cols.remove('scrape')
    
    writer = csv.DictWriter(csvfile, cols)
    writer.writeheader()
    for i in data.rewind():
        i.pop('_id')
        i.pop('scrape')
        writer.writerow(i)